# IMC LUAD

In this notebook we demonstrate how mosna can be used to analyze spatiallly resolved omics data.  
The data used is from the publication by [Sorin et al., Nature, 2023](https://doi.org/10.1038/s41586-022-05672-3) "Single-cell spatial landscapes of the lung tumour immune microenvironment".  
Here tumor cores from 416 patients with lung adenocarcinoma (LUAD) were processed with the [Imaging Mass Cytometry](https://doi.org/10.1038/s43018-020-0026-6) method (cytometry by time of fligh, CyTOF) to produce maps of 35 proteins and study the relationship between cell types in tumors with respect to survival.  

## Imports and data loading

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from time import time
import warnings
import joblib
from pathlib import Path
from time import time
from tqdm import tqdm
import copy
import matplotlib as mpl
import napari
import colorcet as cc
import composition_stats as cs
from sklearn.impute import KNNImputer

from tysserand import tysserand as ty
from mosna import mosna

import matplotlib as mpl
mpl.rcParams["figure.facecolor"] = 'white'
mpl.rcParams["axes.facecolor"] = 'white'
mpl.rcParams["savefig.facecolor"] = 'white'

In [2]:
# If need to reload modules after their modification
from importlib import reload
ty = reload(ty)
mosna = reload(mosna)

In [3]:
RUN_LONG = False

### Objects data

Load files that contains all the detected objects (the cells) across all samples and clinical data.  
Data is available here: https://zenodo.org/records/7760826

In [4]:
data_dir = Path("../data/raw/IMC_LUAD_Sorin_2023")
# try:
#     objects_path = data_dir / "41467_2021_26974_MOESM3_ESM_-_Objects.parquet"
#     obj = pd.read_parquet(data_dir / "41467_2021_26974_MOESM3_ESM_-_Objects.parquet")
# except FileNotFoundError:
#     objects_path = data_dir / "41467_2021_26974_MOESM3_ESM_-_Objects.xlsx"
#     obj = pd.read_excel(objects_path, skiprows=2)
#     # for latter use
#     obj.to_parquet(data_dir / "41467_2021_26974_MOESM3_ESM_-_Objects.parquet")
# obj

In [41]:
import scipy.io
mat = scipy.io.loadmat(str(data_dir / 'LUAD_IMC_CellType' / 'LUAD_D001.mat'))
print("mat.keys():\n", mat.keys())

var_name = 'Boundaries'
print(f'\n------ {var_name} ------')
print(f"type(mat['{var_name}']):", type(mat[var_name]))
print(f"mat['{var_name}'].shape:", mat[var_name].shape)
print(f"type(mat['{var_name}'][0]):", type(mat[var_name][0]))
print(f"mat['{var_name}'][0].shape:", mat[var_name][0].shape)
print(f"type(mat['{var_name}'][0][0]):", type(mat[var_name][0][0]))
print(f"mat['{var_name}'][0][0].shape:", mat[var_name][0][0].shape)
# ==> This is still a mystery

var_name = 'cellTypes'
print(f'\n------ {var_name} ------')
print(f"type(mat['{var_name}']):", type(mat[var_name]))
print(f"mat['{var_name}'].shape:", mat[var_name].shape)
print(f"type(mat['{var_name}'][0]):", type(mat[var_name][0]))
print(f"mat['{var_name}'][0].shape:", mat[var_name][0].shape)
# napari.view_image(mat[var_name], contrast_limits=[mat[var_name].min(), mat[var_name].max()]);
# ==> This is an array storing assigned cell types

var_name = 'allLabels'
print(f'\n------ {var_name} ------')
print(f"type(mat['{var_name}']):", type(mat[var_name]))
print(f"mat['{var_name}'].shape:", mat[var_name].shape)
print(f"type(mat['{var_name}'][0]):", type(mat[var_name][0]))
print(f"mat['{var_name}'][0].shape:", mat[var_name][0].shape)
# napari.view_image(mat[var_name], contrast_limits=[mat[var_name].min(), mat[var_name].max()]);
# ==> This is the array of unique cell types

var_name = 'allCols'
print(f'\n------ {var_name} ------')
print(f"type(mat['{var_name}']):", type(mat[var_name]))
print(f"mat['{var_name}'].shape:", mat[var_name].shape)
print(f"type(mat['{var_name}'][0]):", type(mat[var_name][0]))
print(f"mat['{var_name}'][0].shape:", mat[var_name][0].shape)
# napari.view_image(mat[var_name], contrast_limits=[mat[var_name].min(), mat[var_name].max()]);
# ==> This is probably the coordinates, of cells or nuclei, I don't know if it's x/y/z or z/y/x

mat.keys():
 dict_keys(['__header__', '__version__', '__globals__', 'Boundaries', 'cellTypes', 'allLabels', 'allCols'])

------ Boundaries ------
type(mat['Boundaries']): <class 'numpy.ndarray'>
mat['Boundaries'].shape: (1, 4517)
type(mat['Boundaries'][0]): <class 'numpy.ndarray'>
mat['Boundaries'][0].shape: (4517,)
type(mat['Boundaries'][0][0]): <class 'numpy.ndarray'>
mat['Boundaries'][0][0].shape: (22, 1)

------ cellTypes ------
type(mat['cellTypes']): <class 'numpy.ndarray'>
mat['cellTypes'].shape: (4517, 1)
type(mat['cellTypes'][0]): <class 'numpy.ndarray'>
mat['cellTypes'][0].shape: (1,)

------ allLabels ------
type(mat['allLabels']): <class 'numpy.ndarray'>
mat['allLabels'].shape: (1, 17)
type(mat['allLabels'][0]): <class 'numpy.ndarray'>
mat['allLabels'][0].shape: (17,)

------ allCols ------
type(mat['allCols']): <class 'numpy.ndarray'>
mat['allCols'].shape: (18, 3)
type(mat['allCols'][0]): <class 'numpy.ndarray'>
mat['allCols'][0].shape: (3,)


In [8]:
import tifffile
img = tifffile.tifffile.imread(data_dir / 'LUAD_IMC_MaskTif' / 'LUAD_D002.tif')
print(img.shape)
import napari
napari.view_image(img);

(18, 993, 1007)


In [49]:
mat = scipy.io.loadmat(str(data_dir / 'LUAD_IMC_Segmentation' / 'LUAD_D001' / 'allRegionIndices.mat'))
print("mat.keys():\n", mat.keys())

var_name = 'allRegInds'
print(f'\n------ {var_name} ------')
print(f"type(mat['{var_name}']):", type(mat[var_name]))
print(f"mat['{var_name}'].shape:", mat[var_name].shape)
print(f"type(mat['{var_name}'][0]):", type(mat[var_name][0]))
print(f"mat['{var_name}'][0].shape:", mat[var_name][0].shape)
print(f"type(mat['{var_name}'][0][0]):", type(mat[var_name][0][0]))
print(f"mat['{var_name}'][0][0].shape:", mat[var_name][0][0].shape)
# napari.view_image(mat[var_name], contrast_limits=[mat[var_name].min(), mat[var_name].max()]);
# print(f"mat['{var_name}']: \n", mat[var_name][0][0])
# ==> This is still a mystery

mat.keys():
 dict_keys(['__header__', '__version__', '__globals__', 'allRegInds'])

------ allRegInds ------
type(mat['allRegInds']): <class 'numpy.ndarray'>
mat['allRegInds'].shape: (4517, 1)
type(mat['allRegInds'][0]): <class 'numpy.ndarray'>
mat['allRegInds'][0].shape: (1,)
type(mat['allRegInds'][0][0]): <class 'numpy.ndarray'>
mat['allRegInds'][0][0].shape: (86, 1)


In [50]:
mat = scipy.io.loadmat(str(data_dir / 'LUAD_IMC_Segmentation' / 'LUAD_D001' / 'nuclei_multiscale.mat'))
print("mat.keys():\n", mat.keys())

var_name = 'nucleiImage'
print(f'\n------ {var_name} ------')
print(f"type(mat['{var_name}']):", type(mat[var_name]))
print(f"mat['{var_name}'].shape:", mat[var_name].shape)
print(f"type(mat['{var_name}'][0]):", type(mat[var_name][0]))
print(f"mat['{var_name}'][0].shape:", mat[var_name][0].shape)
# napari.view_image(mat[var_name], contrast_limits=[mat[var_name].min(), mat[var_name].max()]);
# ==> This is an image of nuclei

var_name = 'Boundaries'
print(f'\n------ {var_name} ------')
print(f"type(mat['{var_name}']):", type(mat[var_name]))
print(f"mat['{var_name}'].shape:", mat[var_name].shape)
print(f"type(mat['{var_name}'][0]):", type(mat[var_name][0]))
print(f"mat['{var_name}'][0].shape:", mat[var_name][0].shape)
print(f"type(mat['{var_name}'][0][0]):", type(mat[var_name][0][0]))
print(f"mat['{var_name}'][0][0].shape:", mat[var_name][0][0].shape)
# ==> This is still a mystery

var_name = 'occupancy_image'
print(f'\n------ {var_name} ------')
print(f"type(mat['{var_name}']):", type(mat[var_name]))
print(f"mat['{var_name}'].shape:", mat[var_name].shape)
print(f"type(mat['{var_name}'][0]):", type(mat[var_name][0]))
print(f"mat['{var_name}'][0].shape:", mat[var_name][0].shape)
# napari.view_image(mat[var_name], contrast_limits=[mat[var_name].min(), mat[var_name].max()]);
# ==> This is a binary image of nuclei


mat.keys():
 dict_keys(['__header__', '__version__', '__globals__', 'nucleiImage', 'Boundaries', 'occupancy_image', 'nucleiOccupancyIndexed'])

------ nucleiImage ------
type(mat['nucleiImage']): <class 'numpy.ndarray'>
mat['nucleiImage'].shape: (993, 1007)
type(mat['nucleiImage'][0]): <class 'numpy.ndarray'>
mat['nucleiImage'][0].shape: (1007,)

------ Boundaries ------
type(mat['Boundaries']): <class 'numpy.ndarray'>
mat['Boundaries'].shape: (1, 4517)
type(mat['Boundaries'][0]): <class 'numpy.ndarray'>
mat['Boundaries'][0].shape: (4517,)
type(mat['Boundaries'][0][0]): <class 'numpy.ndarray'>
mat['Boundaries'][0][0].shape: (22, 1)

------ occupancy_image ------
type(mat['occupancy_image']): <class 'numpy.ndarray'>
mat['occupancy_image'].shape: (993, 1007)
type(mat['occupancy_image'][0]): <class 'numpy.ndarray'>
mat['occupancy_image'][0].shape: (1007,)


In [71]:
clinical_path = data_dir / "LUAD Clinical Data.xlsx"
clin = pd.read_excel(clinical_path, index_col=0)
clin

,"Sex (Male: 0, Female: 1)","Age (<75: 0, ≥75: 1)","BMI (<30: 0, ≥30: 1)","Smoking Status (Smoker: 0, Non-smoker:1)","Pack Years (1-30: 0, ≥30: 1)","Stage (I-II: 0, III-IV:1)","Progression (No: 0, Yes: 1)","Death (No: 0, Yes: 1)",Survival or loss to follow-up (years),"Predominant histological pattern (Lepidic:1, Papillary: 2, Acinar: 3, Micropapillary: 4, Solid: 5)"
Key,,,,,,,,,,
LUAD_D001,0,0,1,0.0,1.0,0.0,0.0,1,8.386,3
LUAD_D002,0,0,0,0.0,1.0,0.0,1.0,0,10.111,1
LUAD_D003,0,0,0,0.0,1.0,0.0,0.0,1,2.094,5
LUAD_D004,0,1,0,0.0,1.0,0.0,0.0,1,1.755,4
LUAD_D005,1,0,1,0.0,1.0,0.0,0.0,1,7.598,3
...,...,...,...,...,...,...,...,...,...,...
LUAD_D412,1,0,1,0.0,0.0,0.0,0.0,0,9.944,3
LUAD_D413,1,0,0,0.0,0.0,0.0,0.0,0,8.132,1
LUAD_D414,1,0,0,0.0,1.0,0.0,0.0,0,8.550,3
